In [1]:
import os, json
import pandas as pd
import requests

In [2]:
input_dir = os.getenv('ANCHORE_SCAN_REPORTS_DIR', default = '../../.bob/va-reports/anchore')
output_dir = os.getenv('ANCHORE_ANALYSIS_REPORTS_DIR', default ='../../.bob/va-reports/analysis-reports/anchore/')

print(input_dir)
print(output_dir)

../../.bob/va-reports/anchore
../../.bob/va-reports/analysis-reports/anchore/


In [ ]:
from datetime import datetime
runDate= datetime.today().strftime('%Y%m%d')
runDate

In [3]:
json_files = [os.path.join(dp, f) 
            for dp, dn, filenames in os.walk(input_dir) 
                for f in filenames if f.endswith('-sbom.json')]
print(json_files)
no_of_files=len(json_files)

if no_of_files==0:
    raise Exception('SBOM scan reports not found.. Cannot proceed')

print(no_of_files)

['../../.bob/va-reports/anchore/internal-reports/jupyterhub_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/kubernetes-modifier_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/mxe-deployer-init_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/mxe-deployer-service_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/default-backend_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/model-service_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/model-catalogue-service_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/keycloak-init_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/nifi-service_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/nifi-base_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/pypi-eea_2.2.0-167-sbom.json', '../../.bob/va-reports/anchore/internal-reports/eric-mxe-ga

In [22]:
def load_json(json_file):
    with open(json_file) as jsonReport:
        json_data= json.load(jsonReport)
        os_info={}
        os_info["image"]= json_data["source"]["target"]["userInput"]
        os_info["distro"] = json_data["distro"]
    return os_info

def json_to_df(json_file):
    os_info=load_json(json_file)
    df=pd.json_normalize(os_info)
    df.fillna('', inplace=True)
    return df

dfs=[json_to_df(json_file) for json_file in json_files]
df=pd.concat(dfs)
df.head()


,image,distro.prettyName,distro.name,distro.id,distro.idLike,distro.version,distro.versionID,distro.homeURL,distro.supportURL,distro.bugReportURL,distro.privacyPolicyURL,distro.cpeName
0,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,Ubuntu 20.04.3 LTS,Ubuntu,ubuntu,[debian],20.04.3 LTS (Focal Fossa),20.04,https://www.ubuntu.com/,https://help.ubuntu.com/,https://bugs.launchpad.net/ubuntu/,https://www.ubuntu.com/legal/terms-and-policie...,NaN
0,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,SUSE Linux Enterprise Server 15 SP3,SLES,sles,[suse],15-SP3,15.3,NaN,NaN,NaN,NaN,cpe:/o:suse:sles:15:sp3
0,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,SUSE Linux Enterprise Server 15 SP3,SLES,sles,[suse],15-SP3,15.3,NaN,NaN,NaN,NaN,cpe:/o:suse:sles:15:sp3
0,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,SUSE Linux Enterprise Server 15 SP3,SLES,sles,[suse],15-SP3,15.3,NaN,NaN,NaN,NaN,cpe:/o:suse:sles:15:sp3
0,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,SUSE Linux Enterprise Server 15 SP3,SLES,sles,[suse],15-SP3,15.3,NaN,NaN,NaN,NaN,cpe:/o:suse:sles:15:sp3


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 0
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   image                    68 non-null     object
 1   distro.prettyName        64 non-null     object
 2   distro.name              64 non-null     object
 3   distro.id                64 non-null     object
 4   distro.idLike            57 non-null     object
 5   distro.version           58 non-null     object
 6   distro.versionID         64 non-null     object
 7   distro.homeURL           20 non-null     object
 8   distro.supportURL        14 non-null     object
 9   distro.bugReportURL      20 non-null     object
 10  distro.privacyPolicyURL  13 non-null     object
 11  distro.cpeName           44 non-null     object
dtypes: object(12)
memory usage: 6.9+ KB


In [24]:
with pd.ExcelWriter(f"{output_dir}/os_details_{runDate}.xlsx") as excelWriter: 
    df.to_excel(excelWriter, sheet_name="base_os", index=False)
    